Importing modules

In [154]:
import pandas as pd
import json
import csv
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import import_ipynb
import Similar_Restaurants as sr

In [155]:
raw_data = pd.read_csv('~/Documents/6242 Project/fall2018cse6242/data/arizona_business_details.csv', skipinitialspace=True)

FileNotFoundError: File b'/Users/prashanth/Documents/6242 Project/fall2018cse6242/data/arizona_business_details.csv' does not exist

In [156]:
raw_data['postal_code'] = raw_data['postal_code'].fillna(0)

raw_data['postal_code'] = raw_data['postal_code'].astype(int)


In [157]:
attributes = set()
for each in raw_data['attributes']:
    line = {}
    try:
        each = eval(each)
    except TypeError:
        continue
    for k in each.keys():
        attributes.add(k)

attributes = list(attributes)

In [158]:
def get_attributes(zipcode):

    attribute_count = defaultdict(int)
    data = raw_data[raw_data['postal_code'] == zipcode]
    n = data.shape[0]
    for each in data['attributes']:
        try:
            for attribute in attributes:
                if attribute in each:
                    attribute_count[attribute] += 1
        except TypeError:
            continue
    count_df = pd.DataFrame.from_dict(attribute_count, orient='index', columns= ['count'])
    count_df['percent_of_total'] = count_df['count']/n
    count_df = count_df.sort_values(by = 'percent_of_total', inplace=False, ascending=False)

    return count_df

Building the daframe for regression

In [159]:
features = ['BusinessAcceptsCreditCards',
 'RestaurantsTakeOut',
 'RestaurantsGoodForGroups',
 'GoodForKids',
 'RestaurantsDelivery',
 'RestaurantsReservations',
 'OutdoorSeating',
 'Alcohol',
 'RestaurantsTableService',
 'HasTV',
 'WiFi',
 'BikeParking',
 'Caters',
 'WheelchairAccessible']

check = ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'none', 'False', 'False', 'no', 
         'False', 'False', 'False']

nest_features = ['Ambience', 'GoodForMeal', 'BusinessParking']
add_features = {}
add_features['Ambience'] = ['romantic', 'intimate', 'classy', 'hipster', 'divey', 'touristy', 'trendy', 'upscale', 'casual']
add_features['GoodForMeal'] = ['dessert', 'latenight', 'lunch', 'dinner', 'breakfast', 'brunch']
add_features['BusinessParking'] = ['garage', 'street', 'validated', 'lot', 'valet']



In [160]:
all_features = features + nest_features

In [161]:
def attribute_check(attribute, restaurant, check):
    if attribute in restaurant:
        if restaurant[attribute] == check:
            return 0
        else:
            return 1
    else:
        return 0

In [9]:
def create_dataframe(zipcode):
    attributes_zip = get_attributes(zipcode)
    attributes = attributes_zip[attributes_zip['percent_of_total'] > 0.45].index.values.tolist()

#     print (list(set(attributes) - set(all_features)))
    
    if 'RestaurantsPriceRange2' in attributes:
        attributes.remove('RestaurantsPriceRange2')
    data = raw_data[raw_data['postal_code'] == zipcode]
    dframe = pd.DataFrame()
    for index, row in data.iterrows():

        try:
            each = eval(row['attributes'])
            line = {}
            line['rating'] = row['stars']
            line['biz_id'] = row['business_id']
#             line['zipcode'] = row['postal_code']
            for i in range(len(features)):
                line[features[i]] = attribute_check(features[i], each, check[i])

            # adding the nest features 
            for feature in nest_features:
                if feature in each: 
                    for cat in add_features[feature]:
                        if ast.literal_eval(each[feature])[cat] == True:
                            line[feature + '_' + cat] = 1
                        else:
                            line[feature + '_' + cat] = 0
                else:
                    for cat in add_features[feature]:
                            line[feature + '_' + cat] = 0
                    

            # Adding Noiselevel Data 
#             if 'NoiseLevel' in each:
#                 line['NoiseLevel'] = each['NoiseLevel']
#             else:
#                 line['NoiseLevel'] = 'missing'

    #         # Adding RestaurantsAttire data
    #         if 'RestaurantsAttire' in each:
    #             line['RestaurantsAttire'] = each['RestaurantsAttire']
    #         else:
    #             line['RestaurantsAttire'] = 'missing'


            # converting the dict as df 
            temp_df = pd.DataFrame([line])
            dframe = dframe.append(temp_df, ignore_index=True, sort=False)

        except TypeError:
            continue
    return dframe

In [167]:
len(create_dataframe(85007).columns)



36

In [11]:
create_dataframe(85010)

""


Linear Regression Model Building

In [13]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split


def zip_attributes(zipcode):
    attributes_list = {} 

    dframe = create_dataframe(zipcode)
    if len(dframe) > 0:
        dframe = dframe.drop(columns=['biz_id'])
        dframe = dframe.fillna(0)
    
    
        train, test = train_test_split(dframe, test_size=0)

        X_train = train.loc[:, dframe.columns != 'rating']
        y_train = train['rating']

        X_test = test.loc[:, dframe.columns != 'rating']
        y_test = test['rating']

        # Create linear regression object
        regr = linear_model.LinearRegression()

        # Train the model using the training sets
        regr.fit(X_train, y_train)

        # Make predictions using the testing set
        # y_pred = regr.predict(X_test)

        # # The coefficients
        # print('Coefficients: \n', regr.coef_)
        # # The mean squared error
        # print("Mean squared error: %.2f"
        #       % mean_squared_error(y_test, y_pred))
        # # Explained variance score: 1 is perfect prediction
        # print('Variance score: %.2f' % r2_score(y_test, y_pred))

        # # Plot outputs
        # plt.plot(X_test, y_pred, color='blue', linewidth=3)

        # plt.xticks(())
        # plt.yticks(())

        # plt.show()

        attributes = list(X_train.columns)
        imp_attributes = defaultdict()
        for i in range(len(attributes)):
            if regr.coef_[i] >0:
                imp_attributes[attributes[i]] = regr.coef_[i]


        dic_attributes = dict(sorted(imp_attributes.items(), key=lambda kv: -kv[1])[:8])

        for d in dic_attributes:
            total = len(train)
            count = train[d].sum()
            if count > 0:
                attributes_list[d] = count/total
        attributes_list = dict(sorted(attributes_list.items(), key=lambda kv: -kv[1]))
#         attributes_list['zipcode'] = zipcode
#     print(zipcode)
    return attributes_list

In [14]:
# zipcodes = range(85001, 85005)

zipcodes = [85323,
 85007,
 85033,
 85023,
 85253,
 85257,
 85305,
 85258,
 85204,
 85338,
 85297,
 85044,
 85281,
 85001,
 85032,
 85331,
 85383,
 85224,
 85018,
 85306,
 85282,
 85286,
 85022,
 85210,
 85028,
 85250,
 85395,
 85296,
 85302,
 85048,
 85382,
 85207,
 85259,
 85008,
 85051,
 85202,
 85255,
 85212,
 85024,
 85283,
 85260,
 85003,
 85029,
 85351,
 85340,
 85251,
 85215,
 85019,
 85006,
 85201,
 85284,
 85034,
 85085,
 85020,
 85303,
 85295,
 85013,
 85225,
 85308,
 85256,
 85304,
 85233,
 85353,
 85203,
 85226,
 85326,
 85254,
 85054,
 85262,
 85266,
 85073,
 85307,
 85385,
 85009,
 85035,
 85043,
 85208,
 85015,
 85545,
 85381,
 85004,
 85361,
 85064,
 85031,
 85012,
 85016,
 85234,
 85248,
 85213,
 85017,
 85042,
 85037,
 85206,
 85268,
 85236,
 85378,
 85379,
 85086,
 85301,
 85374,
 85027,
 85249,
 85053,
 85205,
 85298,
 85050,
 85392,
 85375,
 85335,
 85388,
 85363,
 85209,
 85014,
 85287,
 85339,
 85377,
 85040,
 85143,
 85021,
 85041,
 85083,
 85345,
 85821,
 85310,
 85045,
 85058,
 85394,
 85373,
 85087,
 85005,
 85327,
 85318,
 85396,
 85329,
 85311,
 85355,
 85324,
 85264,
 85142,
 85010,
 85252,
 85140,
 85280,
 84295]

In [15]:
zipcode_attributes = {}
for zipcode in zipcodes:
    zipcode_attributes[zipcode] = zip_attributes(zipcode)


In [110]:
zipcode_attributes[85034]

{'WheelchairAccessible': 0.36419753086419754,
 'OutdoorSeating': 0.16666666666666666,
 'Caters': 0.16049382716049382,
 'RestaurantsDelivery': 0.07407407407407407,
 'BusinessParking_street': 0.037037037037037035,
 'Ambience_trendy': 0.012345679012345678,
 'BusinessParking_valet': 0.006172839506172839}

In [ ]:
df = pd.DataFrame.from_dict(zipcode_attributes)
df.to_csv('zipcode_attributes.csv')

In [84]:
def is_the_attribute_present(biz_id, attribute):
    zipcode = raw_data[raw_data['business_id'] == biz_id]['postal_code'].iloc[0]
    df = create_dataframe(zipcode)
    restaurant = df[df['biz_id'] == biz_id]
    val = restaurant[attribute].iloc[0]
    return val

In [152]:
def restaurant_case(biz_id):
    zipcode = raw_data[raw_data['business_id'] == biz_id]['postal_code'].iloc[0]
    df = create_dataframe(zipcode)
    restaurant = df[df['biz_id'] == biz_id]
#     print(restaurant)
    imp_attributes = zipcode_attributes[zipcode]
    imp_attributes_list = list(imp_attributes.keys())
    sug_restaurants = sr.similar_restaurants(biz_id, 10)
    
    existing_attributes = []
    for attr in list(restaurant.columns):
        if (restaurant[attr].iloc[0] == 1) :
            existing_attributes.append(attr)
#     print(existing_attributes)

    
    req_attributes = set(imp_attributes_list) - set(existing_attributes)
    
    values = list(imp_attributes.values())
    range_ = max(values) - min(values);
    min_ = min(values)

    final_attr = defaultdict(dict)
    sim_rests = defaultdict(dict)

    for attribute in list(req_attributes):
        importance = imp_attributes[attribute]
        importance = (importance - min_)/range_
        importance = np.round((importance * 4) + 1)
        
        ind = list(req_attributes).index(attribute)
        final_attr[ind]['attribute'] = attribute
        final_attr[ind]['importance'] = importance
        final_attr[ind]['percentage'] = np.round(imp_attributes[attribute] * 100)
        
        i = 0
        for restaurant in sug_restaurants:
            rest_details = defaultdict(dict)
            if is_the_attribute_present(restaurant, attribute) == 1:
                rest_details['name'] = restaurant
                rest_details['location'] = raw_data[raw_data['business_id'] == restaurant]['postal_code'].iloc[0]
                rest_details['rating'] = raw_data[raw_data['business_id'] == restaurant]['stars'].iloc[0]
                rest_details['review_count'] = raw_data[raw_data['business_id'] == restaurant]['review_count'].iloc[0]
                rest_details['url'] = 'https://www.yelp.com/biz/'+ restaurant
                sim_rests[attribute][i] = rest_details
                i += 1
        
        
    if len(final_attr) > 0:
        final_attr = final_attr
    else:
        final_attr = "hey, seems like you are doing pretty good! kepp it up!!"
    
        
    
    return final_attr, sim_rests


In [66]:
sr.similar_restaurants('_c3ixq9jYKxhLUB0czi0ug', 5)

['Bonny62rswQQ6-sC1DpOrA',
 '1kDS1homhHy0gr5N95oGzw',
 'dnBQZpWTyJzTe0QJf7YK5w',
 'Sg36zmOvyPAmuz6vME56Vw',
 'dq4c_0btrrdcz_Nq6RV02g']

In [69]:
biz_id = '_c3ixq9jYKxhLUB0czi0ug'


In [61]:
restaurant

,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,...,OutdoorSeating,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WheelchairAccessible,WiFi,biz_id,rating
0,1,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,_c3ixq9jYKxhLUB0czi0ug,4.0


In [153]:
restaurant_case('_c3ixq9jYKxhLUB0czi0ug')

(defaultdict(dict,
             {0: {'attribute': 'BusinessParking_street',
               'importance': 2.0,
               'percentage': 22.0},
              1: {'attribute': 'BusinessParking_garage',
               'importance': 1.0,
               'percentage': 2.0},
              2: {'attribute': 'GoodForMeal_dessert',
               'importance': 1.0,
               'percentage': 2.0},
              3: {'attribute': 'Caters',
               'importance': 3.0,
               'percentage': 44.0}}),
 defaultdict(dict,
             {'BusinessParking_street': {0: defaultdict(dict,
                           {'name': 'R3B_ibZvlrgzch_UkYQkrg',
                            'location': 85004,
                            'rating': 3.0,
                            'review_count': 131,
                            'url': 'https://www.yelp.com/biz/R3B_ibZvlrgzch_UkYQkrg'}),
               1: defaultdict(dict,
                           {'name': 'VRTQMh7wVgmzg96YttL3mQ',
                        

In [117]:
sr.similar_restaurants('VRTQMh7wVgmzg96YttL3mQ', 10)

['fmyo8-JBOu5bjczEsLJyBA',
 '5nWfU-qheLL-JzwYgHe_cQ',
 'rqYOFPPrdwMYvSRpI4r3-g',
 'NUh5L9ZX-YrM5uAKhKQXoQ',
 'JRo3APL5f4-TEkuI-SoDcA',
 'R3B_ibZvlrgzch_UkYQkrg',
 'HJFKqzhV4n4ZWepneOzFhQ',
 'l8tSrCIwIyRMBHUiZ9By4A',
 'Bonny62rswQQ6-sC1DpOrA',
 'poX-2lODvtmkR6pve8FXFA']

In [150]:
attributes = ['BusinessParking_street', 'BusinessParking_garage', 'GoodForMeal_dessert', 'Caters']

sug_restaurants = sr.similar_restaurants(biz_id, 10)
sim_rests = defaultdict(dict)
for attribute in attributes:
    i = 0
    for restaurant in sug_restaurants:
        rest_details = defaultdict(dict)
        if is_the_attribute_present(restaurant, attribute) == 1:
            rest_details['name'] = restaurant
            rest_details['location'] = raw_data[raw_data['business_id'] == restaurant]['postal_code'].iloc[0]
            rest_details['rating'] = raw_data[raw_data['business_id'] == restaurant]['stars'].iloc[0]
            rest_details['review_count'] = raw_data[raw_data['business_id'] == restaurant]['review_count'].iloc[0]
            rest_details['url'] = 'https://www.yelp.com/biz/'+ restaurant
            sim_rests[attribute][i] = rest_details
            i += 1

In [151]:
sim_rests

defaultdict(dict,
            {'BusinessParking_street': {0: defaultdict(dict,
                          {'name': 'R3B_ibZvlrgzch_UkYQkrg',
                           'location': 85004,
                           'rating': 3.0,
                           'review_count': 131,
                           'url': 'https://www.yelp.com/biz/R3B_ibZvlrgzch_UkYQkrg'}),
              1: defaultdict(dict,
                          {'name': 'VRTQMh7wVgmzg96YttL3mQ',
                           'location': 85014,
                           'rating': 3.0,
                           'review_count': 242,
                           'url': 'https://www.yelp.com/biz/VRTQMh7wVgmzg96YttL3mQ'})},
             'BusinessParking_garage': {0: defaultdict(dict,
                          {'name': 'R3B_ibZvlrgzch_UkYQkrg',
                           'location': 85004,
                           'rating': 3.0,
                           'review_count': 131,
                           'url': 'https://www.yelp.com/biz

In [36]:
sr.similar_restaurants('_c3ixq9jYKxhLUB0czi0ug', 10)

['Bonny62rswQQ6-sC1DpOrA',
 '1kDS1homhHy0gr5N95oGzw',
 'dnBQZpWTyJzTe0QJf7YK5w',
 'Sg36zmOvyPAmuz6vME56Vw',
 'dq4c_0btrrdcz_Nq6RV02g',
 'HJFKqzhV4n4ZWepneOzFhQ',
 'R3B_ibZvlrgzch_UkYQkrg',
 '0zr7FB0nnZXJmwb94yTbJQ',
 'VRTQMh7wVgmzg96YttL3mQ',
 '5nWfU-qheLL-JzwYgHe_cQ']

In [162]:
dframe.head()

NameError: name 'dframe' is not defined

In [48]:
all_biz = raw_data['business_id']

In [51]:
all_biz.to_csv('all_biz.csv')

In [52]:
raw_data.head()

,Unnamed: 0,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,8,1440 N. Dysart Ave,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants",Avondale,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.448106,-112.341302,Filiberto's Mexican Food,NaN,85323,40,2.5,AZ
1,12,2801 N 15th Ave,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,33.479807,-112.091188,Original Hamburger Works,NaN,85007,277,4.0,AZ
2,22,4131 N 83rd Ave,"{'Ambience': ""{'romantic': False, 'intimate': ...",3ByGQOVgds2YEu6kzl-XEQ,"Nightlife, Bars, American (Traditional), Sport...",Phoenix,NaN,1,33.495298,-112.236076,Rib Shop,NaN,85033,3,2.5,AZ
3,24,7745 W Thomas Rd,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...",CeuTRtwsq6w5rztGOyNMPg,"Mexican, Restaurants",Phoenix,NaN,1,33.479671,-112.225173,Salsitas,NaN,85033,14,2.5,AZ
4,39,4145 N 83Rd,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",AcGRSWCpb7YB95MTsHlGEw,"Pizza, Fast Food, Restaurants",Phoenix,"{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",1,33.495422,-112.235798,Little Caesars Pizza,NaN,85033,4,2.0,AZ


In [135]:
raw_data.sample(7)['business_id'].tolist()

['Todrg1F247iuDIm784XHfA',
 '05BpxQYVbywxliQ9yA0CKw',
 'oeK3WcI-648Tt8prR_OpSg',
 'k-NEUIXaPnXtrzCo1MeO7g',
 'DGaW-qWZc2aHYinHdAT7lQ',
 'utc3FUSX4mPErcRcoTkFYg',
 'H-gllDEH0DHqE-HmzdHtKg']

In [148]:
testset = ['JfHXzulF6yIKgA22YYPedw',
 'L_m-d0Ftm6hoXJPH3Xf0-Q',
 'TgjB-wlAVzol6HoDpE8KYg',
 'NwVj3tbEcR2F8fbrcOEV0Q',
 'm2zASwlOL2vecCKHfmGAnQ',
 'hCeDhb2p5L7Nkvgn7WeRCw',
 'p9v9Sl2g_ibaajWvO0NjkQ',
 'L7rjzj2g2EEbHfee9V-X2Q',
 'ERM603jbIbNqX2c7Ww1Qiw',
 'CCs5domumJRZ3-Blpt8jTw',
 'MOi64A9mDpK9zUB8546L8Q',
 'ec9mv9cUMObXqyh3yjgIbA',
 'p_-fGCrpUQZXSO6sGAs6Tg',
 'QAuQV01Yz2Gl89ZN6c1F4w',
 'fT6Uwl6abRQfiNgxre_qbg',
 'rT8DwZu7pvuWKRM1nwVuug',
 'qfr7bHu1-VXUTQWXoSMf9g',
 'CVrySTnC86omXXAm0GJMyA',
 'XF4w6wG1JOvfKZlQcxiWgQ',
 'fKxdy5SiWcSvYrLdxtYdmA',
 'swi7mi1ixWpu5-tZO2mtsg',
 'V0GHF8CkhOovDeQKCn7YQg',
 '5cXym2kMtJgKeaxdF2SrJA',
 '2QcPbyRAw79poY9OKP3hfg',
 'pdWnWH3SnjTTkRZx0y2wSw',
 'b5kC2XepQ1-6qCDpvtoxxA',
 'a6SqJZTFMpSOR6Qb0STyFA',
 'TM3UY0fCq0CQbuskoq0OWQ',
 'vg4ynLZ0yJ89pT4s_CEMzQ',
 'jmRm2wfniXivveq4yI_ADw']


for test in testset:
    
    print(test, list(restaurant_case(test)), '\n')

# restaurant_case('2aeZ9Xm_mjia11jf1pg7QA')
# sr.similar_restaurants('VRTQMh7wVgmzg96YttL3mQ', 10)

JfHXzulF6yIKgA22YYPedw [defaultdict(<class 'dict'>, {0: {'attribute': 'Ambience_trendy', 'importance': 1.0, 'percentage': 3.0}, 1: {'attribute': 'GoodForMeal_latenight', 'importance': 1.0, 'percentage': 4.0}, 2: {'attribute': 'Ambience_intimate', 'importance': 1.0, 'percentage': 2.0}, 3: {'attribute': 'BusinessParking_street', 'importance': 1.0, 'percentage': 3.0}}), defaultdict(<class 'dict'>, {})] 

L_m-d0Ftm6hoXJPH3Xf0-Q [defaultdict(<class 'dict'>, {0: {'attribute': 'WheelchairAccessible', 'importance': 1.0, 'percentage': 44.0}, 1: {'attribute': 'GoodForMeal_lunch', 'importance': 1.0, 'percentage': 45.0}}), defaultdict(<class 'dict'>, {})] 

TgjB-wlAVzol6HoDpE8KYg [defaultdict(<class 'dict'>, {0: {'attribute': 'Ambience_upscale', 'importance': 1.0, 'percentage': 1.0}, 1: {'attribute': 'Ambience_divey', 'importance': 1.0, 'percentage': 8.0}, 2: {'attribute': 'BusinessParking_garage', 'importance': 1.0, 'percentage': 1.0}}), defaultdict(<class 'dict'>, {})] 

NwVj3tbEcR2F8fbrcOEV0Q [

CVrySTnC86omXXAm0GJMyA [defaultdict(<class 'dict'>, {0: {'attribute': 'Ambience_upscale', 'importance': 1.0, 'percentage': 1.0}, 1: {'attribute': 'BusinessParking_garage', 'importance': 1.0, 'percentage': 1.0}, 2: {'attribute': 'Caters', 'importance': 4.0, 'percentage': 47.0}, 3: {'attribute': 'OutdoorSeating', 'importance': 4.0, 'percentage': 47.0}, 4: {'attribute': 'Ambience_divey', 'importance': 1.0, 'percentage': 8.0}, 5: {'attribute': 'BusinessParking_street', 'importance': 2.0, 'percentage': 17.0}}), defaultdict(<class 'dict'>, {})] 

XF4w6wG1JOvfKZlQcxiWgQ [defaultdict(<class 'dict'>, {0: {'attribute': 'Ambience_upscale', 'importance': 1.0, 'percentage': 3.0}, 1: {'attribute': 'Ambience_divey', 'importance': 1.0, 'percentage': 2.0}, 2: {'attribute': 'BusinessParking_valet', 'importance': 1.0, 'percentage': 8.0}, 3: {'attribute': 'Ambience_classy', 'importance': 1.0, 'percentage': 5.0}}), defaultdict(<class 'dict'>, {})] 

fKxdy5SiWcSvYrLdxtYdmA [defaultdict(<class 'dict'>, {0: {